In [1]:
import os
import shutil
import stat
import seaborn as sns
import collections
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.image as img
import random
import cv2
import PIL
import matplotlib.pyplot as plt
import matplotlib.image as img
from os import listdir
from os.path import isfile, join
from collections import defaultdict
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from sklearn.model_selection import train_test_split
from skimage.io import imread
from keras.utils.np_utils import to_categorical
from keras.applications.inception_v3 import preprocess_input
from keras.models import load_model
from shutil import copy
from shutil import copytree, rmtree
import tensorflow.keras.backend 
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow import keras
from tensorflow.keras import models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models
from keras.models import load_model

In [2]:
# Method to load train-test files.
def load_train_test_data(path_to_train_imgs, path_to_test_imgs):
    X_train, y_train = load_images(path_to_train_imgs)
    X_test, y_test = load_images(path_to_test_imgs)
    return X_train, y_train, X_test, y_test

In [30]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import numpy as np

# Clearing the session
tf.keras.backend.clear_session()

# Defining parameters
n_classes = 101
img_width, img_height = 224, 224
train_data_dir = r"C:\Users\kurt_\Desktop\food-101\train"
validation_data_dir = r"C:\Users\kurt_\Desktop\food-101\test"
nb_train_samples = 75750
nb_validation_samples = 25250
batch_size = 32
epochs = 20

# Data generators
def add_gaussian_noise(image):
    noise = np.random.normal(loc=0, scale=0.1, size=image.shape)
    return image + noise

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],  # Rastgele parlaklık
    preprocessing_function=add_gaussian_noise  # Gauss gürültüsü
)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

# MobileNetV2 model
mbv2 = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
x = mbv2.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(n_classes, activation='softmax')(x)

model = Model(inputs=mbv2.input, outputs=predictions)
model.compile(optimizer=Adamax(), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
checkpointer = ModelCheckpoint(filepath='best_model_101class_mobilenetv2den3.hdf5', save_best_only=True)
csv_logger = CSVLogger('history_101class_mobilenetv2den3.log')

# Training the model
history_101class_mobilenetv2den3 = model.fit(train_generator,
                    steps_per_epoch=nb_train_samples // batch_size,
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,
                    epochs=epochs,
                    verbose=1,
                    callbacks=[checkpointer, csv_logger])

# Saving the trained model
model.save('model_trained_101class_mobilenetv2den3.hdf5')
print('Training completed.')


Found 75750 images belonging to 101 classes.
Found 25250 images belonging to 101 classes.
Epoch 1/20
2367/2367 [==============================] - 941s 397ms/step - loss: 2.3664 - accuracy: 0.4391 - val_loss: 2.4431 - val_accuracy: 0.4592
Epoch 2/20
2367/2367 [==============================] - 859s 363ms/step - loss: 1.5563 - accuracy: 0.6092 - val_loss: 2.4908 - val_accuracy: 0.4577
Epoch 3/20
2367/2367 [==============================] - 856s 361ms/step - loss: 1.3187 - accuracy: 0.6634 - val_loss: 1.9697 - val_accuracy: 0.5575
Epoch 4/20
2367/2367 [==============================] - 852s 360ms/step - loss: 1.1544 - accuracy: 0.7010 - val_loss: 1.5789 - val_accuracy: 0.6202
Epoch 5/20
2367/2367 [==============================] - 855s 361ms/step - loss: 1.0477 - accuracy: 0.7248 - val_loss: 1.9896 - val_accuracy: 0.5360
Epoch 6/20
2367/2367 [==============================] - 855s 361ms/step - loss: 0.9541 - accuracy: 0.7484 - val_loss: 1.7413 - val_accuracy: 0.6018
Epoch 7/20
2367/2367 [

In [31]:
model_path = r"C:\Users\kurt_\best_model_101class_mobilenetv2den3.hdf5"
test_data_path = r"C:\Users\kurt_\Desktop\food-101\test"

In [32]:
# Performans metriklerini hesaplamak için fonksiyon tanımla
def calculate_metrics(model, test_gen):
    # Modelden tahminler yap
    predictions = model.predict(test_gen)
    predicted_classes = np.argmax(predictions, axis=1)
    
    # Gerçek sınıfları al
    true_classes = test_gen.classes
    
    # Sınıf etiketlerini al
    class_labels = list(test_gen.class_indices.keys())

    # Doğruluk, hassasiyet, geri çağırma ve F1 puanını hesapla
    accuracy = accuracy_score(true_classes, predicted_classes)
    precision = precision_score(true_classes, predicted_classes, average='weighted')
    recall = recall_score(true_classes, predicted_classes, average='weighted')
    f1 = f1_score(true_classes, predicted_classes, average='weighted')

    return accuracy, precision, recall, f1

In [33]:
# Boyutları tanımla
height, width = 224, 224

# Batch boyutunu tanımla
batch_size = 32

# Test veri üreteci oluştur
test_data_gen = ImageDataGenerator(rescale=1. / 255)
test_gen = test_data_gen.flow_from_directory(test_data_path, target_size=(height, width), batch_size=batch_size, class_mode='categorical', shuffle=False)

# Modeli yükle
model = load_model(model_path, compile=False)

# Performans metriklerini hesapla
accuracy, precision, recall, f1_score = calculate_metrics(model, test_gen)

# Hesaplanan metrikleri yazdır
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


Found 25250 images belonging to 101 classes.
790/790 [==============================] - 67s 84ms/step
Accuracy: 0.6971089108910891
Precision: 0.7263331105641404
Recall: 0.6971089108910891
F1 Score: 0.6995881159377603
